In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/amz_uk_processed_data.csv")

In [4]:
df.head()

,asin,title,imgUrl,productURL,stars,reviews,price,isBestSeller,boughtInLastMonth,categoryName
0,B09B96TG33,"Echo Dot (5th generation, 2022 release) | Big ...",https://m.media-amazon.com/images/I/71C3lbbeLs...,https://www.amazon.co.uk/dp/B09B96TG33,4.7,15308,21.99,False,0,Hi-Fi Speakers
1,B01HTH3C8S,"Anker Soundcore mini, Super-Portable Bluetooth...",https://m.media-amazon.com/images/I/61c5rSxwP0...,https://www.amazon.co.uk/dp/B01HTH3C8S,4.7,98099,23.99,True,0,Hi-Fi Speakers
2,B09B8YWXDF,"Echo Dot (5th generation, 2022 release) | Big ...",https://m.media-amazon.com/images/I/61j3SEUjMJ...,https://www.amazon.co.uk/dp/B09B8YWXDF,4.7,15308,21.99,False,0,Hi-Fi Speakers
3,B09B8T5VGV,"Echo Dot with clock (5th generation, 2022 rele...",https://m.media-amazon.com/images/I/71yf6yTNWS...,https://www.amazon.co.uk/dp/B09B8T5VGV,4.7,7205,31.99,False,0,Hi-Fi Speakers
4,B09WX6QD65,Introducing Echo Pop | Full sound compact Wi-F...,https://m.media-amazon.com/images/I/613dEoF9-r...,https://www.amazon.co.uk/dp/B09WX6QD65,4.6,1881,17.99,False,0,Hi-Fi Speakers


In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("BAAI/bge-small-en-v1.5")

# Our sentences to encode
sentences = [
    "This framework generates embeddings for each input sentence",
    "Sentences are passed as a list of string.",
    "The quick brown fox jumps over the lazy dog."
]

# Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

print(len(embeddings[0]))

# Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

/Users/samuel.matioli/.pyenv/versions/3.9.4/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


384
Sentence: This framework generates embeddings for each input sentence
Embedding: [-3.03472448e-02  1.72387585e-02 -2.10999362e-02 -1.08564170e-02
 -1.58408172e-02  3.37220207e-02 -6.51593655e-02 -1.08955372e-02
  5.49290180e-02  7.54610449e-03 -6.76931515e-02 -1.22491857e-02
  9.18945372e-02  8.05111416e-03  7.32351840e-02  4.40290459e-02
 -2.51056384e-02  3.55514474e-02 -3.84593979e-02 -5.43219000e-02
  7.00224936e-02 -2.71707810e-02  3.05612329e-02 -5.67643195e-02
 -6.59218952e-02  3.47670279e-02 -7.90049508e-03 -2.46839374e-02
 -3.22038494e-02 -1.67230114e-01  2.27734726e-02 -4.01510075e-02
  9.99104604e-02  3.56599092e-02 -2.67715082e-02  2.50015128e-02
 -7.52146766e-02  5.43454923e-02 -1.43130729e-02  1.00398613e-02
  1.34781199e-02  1.00998199e-02 -9.02312715e-03 -4.88470718e-02
  2.68938076e-02 -5.54522239e-02 -6.17890656e-02 -9.42016300e-03
 -2.97371689e-02  3.32471989e-02 -4.83121052e-02 -7.14995712e-02
 -2.91031208e-02  6.67277724e-03  2.05186736e-02 -2.79145432e-04
  8.2

In [6]:
len(embeddings[0])

384

In [ ]:
#df['embedding'] = df['title'].apply(lambda x : model.encode(x, normalize_embeddings=True))

In [7]:
columns_to_keep = ['asin', 'title']
columns_to_drop = df.columns.difference(columns_to_keep)
df.drop(columns=columns_to_drop, inplace=True)

In [8]:
df.head()

,asin,title
0,B09B96TG33,"Echo Dot (5th generation, 2022 release) | Big ..."
1,B01HTH3C8S,"Anker Soundcore mini, Super-Portable Bluetooth..."
2,B09B8YWXDF,"Echo Dot (5th generation, 2022 release) | Big ..."
3,B09B8T5VGV,"Echo Dot with clock (5th generation, 2022 rele..."
4,B09WX6QD65,Introducing Echo Pop | Full sound compact Wi-F...


In [10]:
# Drop rows without the 'title' column
df.dropna(subset=['title'], inplace=True)

# Reset the index
df.reset_index(drop=True, inplace=True)
df.shape

(2222742, 2)

In [11]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime

chunks = []

# Loop through the DataFrame with tqdm
print(f"""Started: {datetime.now()}""")
for index, row in df.iterrows():
    row['emb'] = model.encode(row["title"], normalize_embeddings=True)
    chunks.append(row)
    # Dump every 10,000 records to a CSV file
    if len(chunks) == 10000:
        print(f"""Chunk {index}: {datetime.now()}""")
        # Convert the list of chunks to a DataFrame
        chunk_df = pd.DataFrame(chunks)
        chunk_df['emb'] = chunk_df['emb'].apply(lambda x: ','.join(map(str, x)))
        # Dump to CSV
        chunk_df.to_csv(f'../emb_amz_uk/chunk_{index}.csv', index=False)
        # Clear the chunks list
        chunks.clear()

# Check if there are remaining chunks to dump
if len(chunks) > 0:
    chunk_df = pd.DataFrame(chunks)
    chunk_df.to_csv(f'chunk_{index}.csv', index=False)

Started: 2024-04-03 19:07:41.419159
Chunk 9999: 2024-04-03 19:10:47.372080
Chunk 19999: 2024-04-03 19:14:09.270107
Chunk 29999: 2024-04-03 19:17:25.645154
Chunk 39999: 2024-04-03 19:20:44.584377
Chunk 49999: 2024-04-03 19:23:55.290540
Chunk 59999: 2024-04-03 19:37:12.788574
Chunk 69999: 2024-04-03 20:05:54.832971
Chunk 79999: 2024-04-03 20:57:34.717194
Chunk 89999: 2024-04-03 22:02:39.770922
Chunk 99999: 2024-04-03 22:54:23.647660
Chunk 109999: 2024-04-04 00:02:30.931723
Chunk 119999: 2024-04-04 01:11:17.610871
Chunk 129999: 2024-04-04 02:19:12.860644
Chunk 139999: 2024-04-04 03:09:21.401920
Chunk 149999: 2024-04-04 04:02:18.387876
Chunk 159999: 2024-04-04 05:08:34.474937
Chunk 169999: 2024-04-04 05:41:38.625783
Chunk 179999: 2024-04-04 06:23:04.266279
Chunk 189999: 2024-04-04 06:26:33.049041
Chunk 199999: 2024-04-04 06:29:57.292565
Chunk 209999: 2024-04-04 06:33:19.567687
Chunk 219999: 2024-04-04 06:36:51.731066
Chunk 229999: 2024-04-04 06:40:27.040116
Chunk 239999: 2024-04-04 06:57:5

Chunk 1979999: 2024-04-04 17:01:47.906554
Chunk 1989999: 2024-04-04 17:05:13.283243
Chunk 1999999: 2024-04-04 17:08:43.392570
Chunk 2009999: 2024-04-04 17:12:17.984760
Chunk 2019999: 2024-04-04 17:15:54.207745
Chunk 2029999: 2024-04-04 17:19:30.321541
Chunk 2039999: 2024-04-04 17:23:05.218568
Chunk 2049999: 2024-04-04 17:26:41.435567
Chunk 2059999: 2024-04-04 17:30:07.491899
Chunk 2069999: 2024-04-04 17:33:40.893620
Chunk 2079999: 2024-04-04 17:37:14.441841
Chunk 2089999: 2024-04-04 17:40:47.510577
Chunk 2099999: 2024-04-04 17:44:15.653521
Chunk 2109999: 2024-04-04 17:47:56.675652
Chunk 2119999: 2024-04-04 17:51:44.498911
Chunk 2129999: 2024-04-04 17:55:19.237784
Chunk 2139999: 2024-04-04 17:58:56.505680
Chunk 2149999: 2024-04-04 18:02:28.822949
Chunk 2159999: 2024-04-04 18:06:07.584836
Chunk 2169999: 2024-04-04 18:10:07.796919
Chunk 2179999: 2024-04-04 18:14:04.996818
Chunk 2189999: 2024-04-04 18:17:48.757036
Chunk 2199999: 2024-04-04 18:21:53.937174
Chunk 2209999: 2024-04-04 18:25:46

In [25]:
chunk_df.iloc[0]["emb"]

'-0.05143983,-0.026589058,0.062380653,0.04195266,-0.036118336,0.0037420064,-0.043784473,0.003866413,-0.010216172,-0.017910538,0.0688854,-0.007425397,-0.017913304,0.008687979,0.003623576,-0.010199975,0.029775161,0.04962787,-0.10154397,0.012147993,-0.02020218,-0.020421777,-0.061104737,0.026220089,-0.038472366,0.024007516,-0.017957335,-0.009721564,-0.01941375,-0.06666411,-0.06211478,0.020508545,0.014113841,0.007472432,0.01616603,-0.034530636,-0.05161118,0.025503255,-0.028730463,-0.021641525,-0.026192576,-0.005861912,-0.030439055,-0.07599129,-0.006936844,-0.047952317,-0.054063078,0.027654152,0.0129583115,0.005166324,-0.03066886,-0.008660311,-0.06578653,0.018754568,0.03815604,0.10285456,0.023074271,0.008720535,-0.00012186595,-0.011818375,0.06907888,0.060342956,-0.13113405,0.044521388,-0.07506469,0.0045282263,-0.006433377,-0.034262843,0.023638127,0.058893226,-0.009844009,-0.016950134,0.1555491,0.007171449,0.030740628,0.026954997,0.037245486,-0.05371106,0.0064049615,0.06685357,0.041623063,-0.

In [21]:
import numpy as np
chunk_df['embstr'] = chunk_df['emb'].apply(lambda x: ','.join(map(str, x)))
with np.printoptions(linewidth=100000):
    chunk_df[['asin','title','embstr']].to_csv(f'chunk_000.csv', quotechar="\"",index=False)